In [1]:
from utils import FakeTRTFunc
from model import load_as_trt_model
import chess.syzygy as syzygy

use_fake = True
if use_fake:
    trt_func = FakeTRTFunc()
else:
    trt_func, _ = load_as_trt_model()
tablebase = syzygy.open_tablebase("/home/tomaz/ChessBot_v2/syzygy/3-4-5")

2025-01-14 17:34:02.785955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from selfplay import play_game
from configs import selfplayConfig

_, _, _ = play_game(trt_func, tablebase, 2)



  +---+---+---+---+---+---+---+---+
  | r |   |   | q | k | Q |   |   | 8
  +---+---+---+---+---+---+---+---+
  | p |   |   | p |   |   | p | r | 7
  +---+---+---+---+---+---+---+---+
  | b |   | p |   |   | n |   |   | 6
  +---+---+---+---+---+---+---+---+
  | n | p |   |   |   | p |   | p | 5
  +---+---+---+---+---+---+---+---+
  | P | B |   | P |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   | N |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   | P | P |   | N | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  |   | R |   |   | K |   |   | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r2qkQ2/p2p2pr/b1p2n2/np3p1p/PB1P4/5N2/1PP1NPPP/1R2K2R b Kq - 0 15
Key: 15c38887538a2f8e
White wins by checkmate in 29 moves in 3.98 seconds


In [6]:
from rchess import Board
board = Board("6k1/8/8/8/1B3K2/8/8/3b4 b - - 0 111")
print(board)
print(board.legal_moves())


  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   | k |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   | B |   |   |   | K |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   | b |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 6k1/8/8/8/1B3K2/8/8/3b4 b - - 0 111
Key: 1a8325d150c15122
["g8f7", "g8g7", "g8h7", "g8h8", "d1c2", "d1e2", "d1b3", "d1f3", "d1a4", "d1g4", "d1h5"]


In [10]:
board.terminal()

(True, None)